In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

import sys
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/")
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils/")
import common_functions as common

## Epsilon energy

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
#WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

XLINE = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

In [ ]:
#full_df.groupby(["Type","Part","Success"]).count()
#full_df.groupby(["Type","Part"]).describe()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['Denergy_tc'], df['peSum_tc'], '.b', alpha=0.1, label="tc")
ax.plot(df['Denergy_wt'], df['peSum_wt'], '.r', alpha=0.1, label="wt")

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("e_energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

common.plot_hist2d(ax1, df['Denergy_tc'].dropna(how='any').values,
                   df['peSum_tc'].dropna(how='any').values,
                   "e_energy", "npe", logx=True, logy=True, logz=True)
common.plot_hist2d(ax2, df['Denergy_wt'].dropna(how='any').values,
                   df['peSum_wt'].dropna(how='any').values,
                   "e_energy", "npe", logx=True, logy=True, logz=True)

ax1.set_xlim(0.001, 1.)
ax2.set_xlim(0.001, 1.)

ax1.set_title("tc")
ax2.set_title("wt")